In [3]:
import numpy as np, matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import sys
from scipy import stats
from sklearn.cross_validation import *

%matplotlib inline

In [5]:
matchup_df = pd.read_csv("match")
matchup_df['matchID'] = matchup_df['bID'] + '_' + matchup_df['pID']

IOError: File temp_matchupdf.csv does not exist

In [ ]:
dftouse=matchup_df.copy()

In [ ]:
def compute_supports(df):
    ubids=df.bID.unique()
    pitch = df.groupby('pID').bID.unique()
    bdict={}
    for e,v in zip(pitch.index.values, pitch.values):
        bdict[e] = np.array([item in v for item in ubids])
    pitchers=bdict.keys()
    supports=[]
    for i,p1 in enumerate(pitchers):
        for j,p2 in enumerate(pitchers):
            if  i < j:
                supmask = (bdict[p1] & bdict[p2])
                common_batters = np.sum(supmask)
                supports.append(common_batters)
    print "mean support",np.mean(supports), "median support", np.median(supports)
    return supports, bdict

In [ ]:
def recompute_frame(ldf):
    """
    takes a dataframe ldf, makes a copy of it, and returns the copy
    with all averages and review counts recomputed
    this is used when a frame is subsetted.
    """
    ldfb=ldf.groupby('bID')
    ldfp=ldf.groupby('pID')
    nldf=ldf.copy()
    #Conglomerate pitcher stats
    nldf.set_index(['pID'], inplace=True)
    nldf['ovp_AVG'] = ldfp.H.sum() / ldfp.AB.sum()
    nldf['ovp_FACED']= ldfp.AB.count()
    nldf['ovp_OBP'] = (ldfp.H.sum() + ldfp.W.sum()) / ldfp.PA.sum()
    nldf['ovp_PA'] = ldfp.PA.sum()
    nldf.reset_index(inplace=True)
    #Conglomerate batter stats
    nldf.set_index(['bID'], inplace=True)
    nldf['ov_AVG'] = ldfb.H.sum() / ldfb.AB.sum()
    nldf['ov_FACED']= ldfb.AB.count()
    nldf['ov_OBP'] = (ldfb.H.sum() + ldfb.W.sum()) / ldfb.PA.sum()
    nldf['ov_PA'] = ldfb.PA.sum()
    nldf.reset_index(inplace=True)
    return nldf

In [ ]:
def make_smaller(df, pacountp, pacountb):
    smallidf1=df[(df.ovp_PA > pacountp) & (df.ov_PA > pacountb)]
    smalldf=recompute_frame(smallidf1)
    return smalldf

In [ ]:
def make_smaller_2(df, col, limit, greater=True):
    if greater:
        smallidf = df[(df[col] > limit)]
    else:
        smallidf = df[(df[col] < limit)]
    return recompute_frame(smallidf)

In [ ]:
#Set minimum number of times batter/pitcher faced each other
smalldf = make_smaller_2(dftouse, 'PA', 7)

#Set minimum number of total plate appearances for batter and pitcher
smalldf = make_smaller(smalldf, 400, 400)

smalldf.to_csv('small2.csv', index=False)

In [ ]:
smalldf=smalldf.drop_duplicates(['bID','pID'])

In [ ]:
smalldf[smalldf.ov_FACED > 100].shape

In [ ]:
trainlist=[]
testlist=[]
validatelist=[]
take=21 #11 matchups between validation and test set
for k, v in smalldf.groupby('bID'):
    if len(v) > 100: #batter has faced at least 150 pitchers
        train_rows, test_valid_rows = train_test_split(v, test_size=take)
        trainlist.append(train_rows)
        valid_rows, test_rows = train_test_split(test_valid_rows, test_size=0.4)
        validatelist.append(valid_rows) 
        testlist.append(test_rows) 
    else:
        trainlist.append(v)
traindf=pd.concat(trainlist)
validatedf=pd.concat(validatelist)
testdf=pd.concat(testlist)

In [ ]:
def sim_batters(bID, df):
    